### Text Summarization and Translation using DeepSpeed

In this intermediate-level tutorial we will first look at using ``deepspeed`` to do inference for a text summarization and a text translation task.  We will walk through the code changes and show you how to run the python files.  We include some supplemental (advanced) exercises you can do at the conclusion of our workshop. For the supplemental exercises you will need to use more advanced GPU hardware and a larger cluster.

If you have more questions after the workshop, please reach out to our Domino Data Lab Field Data Science team.  We're glad to help!

#### What the learner will be able to do at the end of this tutorial:

- Execute simple summarization and translation tasks
- Explain how to modify code to run in the DeepSpeed framework
- Understand how to use DeepSpeed to run training pipelines on million+ parameter models
- Assess their model to determine whether training a new model would be helpful
- Leverage the ZeRO3 model using our supplemental exercises


#### Text Summarization Task

Text summarization has long been a popular artificial intelligence technique in which large amounts of language are summarized into simple short phrases.  The original models performing summarization were based on LDA, and performed poorly.  Since that time, many more sophisticated models have been created including several transformer models.  In this example we chose a relatively small transformer model (t5-base).  The original text summarization code (without ``deepspeed`` looks like the code below:

```
import deepspeed
import torch
import os
from transformers import pipeline
from transformers.models.t5.modeling_t5 import T5Block

summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")

```

Very little is required to create a version that runs with ``deepspeed``.  Notice the changes in the code below:

```
import deepspeed
import torch
import os
from transformers import pipeline
from transformers.models.t5.modeling_t5 import T5Block

local_rank = int(os.getenv('LOCAL_RANK', '0'))
world_size = int(os.getenv('WORLD_SIZE', '4'))

# Init summarizer and initialize in deepspeed

summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", device=local_rank)

summarizer.model = deepspeed.init_inference(summarizer.model,
     mp_size=world_size,
     dtype=torch.float,
     injection_policy={T5Block: ('SelfAttention.o', 'EncDecAttention.o', 'DenseReluDense.wo')})

```

The local rank and world size are set based on environmental variables.  Then the inference initialization happens with the deepspeed library in the line ```deepspeed.init_inference(summarizer.model)```.  This simple code change enables the code to run within the ``deepspeed`` framework or with ``mpi``.

Original text:

>Paul Walker is hardly the first actor to die during a production. 

>But Walker's death in November 2013 at the age of 40 after a car crash was especially eerie given his rise to fame in the "Fast and Furious" film franchise. 

>The release of "Furious 7" on Friday offers the opportunity for fans to remember -- and possibly grieve again -- the man that so many have praised as one of the nicest guys in Hollywood. 

>"He was a person of humility, integrity, and compassion," military veteran Kyle Upham said in an email to CNN. 
Walker secretly paid for the engagement ring Upham shopped for with his bride. 

>"We didn't know him personally but this was apparent in the short time we spent with him. 

>I know that we will never forget him and he will always be someone very special to us," said Upham. 
The actor was on break from filming "Furious 7" at the time of the fiery accident, which also claimed the life of the car's driver, Roger Rodas. 
Producers said early on that they would not kill off Walker's character, Brian O'Connor, a former cop turned road racer. Instead, the script was rewritten and special effects were used to finish scenes, with Walker's brothers, Cody and Caleb, serving as body doubles. 
There are scenes that will resonate with the audience -- including the ending, in which the filmmakers figured out a touching way to pay tribute to Walker while "retiring" his character. At the premiere Wednesday night in Hollywood, Walker's co-star and close friend Vin Diesel gave a tearful speech before the screening, saying "This movie is more than a movie." "You'll feel it when you see it," Diesel said. "There's something emotional that happens to you, where you walk out of this movie and you appreciate everyone you love because you just never know when the last day is you're gonna see them." There have been multiple tributes to Walker leading up to the release. Diesel revealed in an interview with the "Today" show that he had named his newborn daughter after Walker. 

>Social media has also been paying homage to the late actor. A week after Walker's death, about 5,000 people attended an outdoor memorial to him in Los Angeles. Most had never met him. Marcus Coleman told CNN he spent almost $1,000 to truck in a banner from Bakersfield for people to sign at the memorial. "It's like losing a friend or a really close family member ... even though he is an actor and we never really met face to face," Coleman said. "Sitting there, bringing his movies into your house or watching on TV, it's like getting to know somebody. It really, really hurts." Walker's younger brother Cody told People magazine that he was initially nervous about how "Furious 7" would turn out, but he is happy with the film. "It's bittersweet, but I think Paul would be proud," he said. CNN's Paul Vercammen contributed to this report.

Now let's run summarization with DeepSpeed.  Keep in mind DeepSpeed does not always give a performance gain.  Its main goal is to enable training and inferences of very large, very memory intensive models.  So as the data and parameters get larger, deepspeed will become more impactful.  DeepSpeed enables the training of models like Yanex which are 100 Billion parameters.  These models can then be used for summarization and translation tasks.  In this notebook we show two trivial examples of performing both tasks.  If you're interested in the Yanex YaML-100B model it is available here: https://github.com/yandex/YaLM-100B.

Our summarization came out to the following:

>Summary: "he was a person of humility, integrity, and compassion," military veteran says . he secretly paid for the engagement ring he shopped for with his bride . the actor was on break from filming "Furious 7" when he died in a car crash .

Not bad, but could be improved.  Can you think of some ways to improve the model inference?  Is there another language model you would try?  See what happens if you choose another model.

In [ ]:
## test to see that your workers are connected

!mpirun hostname

In [ ]:
%%time

## first try it without deepspeed

!python summarization.py

In [ ]:
%%time

## now notice the difference with deepspeed

!deepspeed --num_gpus 1 deepspeed_summarization.py

### A trivial translation task from English to French

>The quick brown fox jumps over the lazy dog.

The text above is translated from english to french.  This simple translation task requires minimal code changes.  For example see the code below:

```

import deepspeed
import torch
import os
from transformers import pipeline
from transformers.models.t5.modeling_t5 import T5Block

local_rank = int(os.getenv('LOCAL_RANK', '0'))
world_size = int(os.getenv('WORLD_SIZE', '4'))

translator = pipeline("translation_en_to_fr", model="t5-base", tokenizer="t5-base", device=local_rank)

translator.model = deepspeed.init_inference(translator.model,
    mp_size=world_size,
    dtype=torch.float,
    injection_policy={T5Block: ('SelfAttention.o', 'EncDecAttention.o', 'DenseReluDense.wo')})

```

Much like with the summarization task there are only a few key variables ( ``local_rank``, ``world_size``) that need to be set. Then the inference initializer should be from the ``deepspeed`` library (``deepseed.init_inference.``).

The output is an English sentence translated to French.  If you speak French, how good was the translation?  

>'translation_text': 'Le renard brun rapide saute au-dessus du chien lazy.'

Google translates gives the following for the same sentence.

>Le renard brun rapide saute par-dessus le chien paresseux.

So not quite the same.  What would you do to improve this translation?  There are several examples in the Hugging Face documentation of changes you could make.  For example you could try the French translator model: [French Translation Trained Model](https://huggingface.co/aschwinghamer/French_Translator).

In [ ]:
%%time

!deepspeed --num_gpus 1 deepspeed_translation.py

#### What we learned in this tutorial:

- Running a summarization or translation model in the ``deepspeed`` framework requires minimal changes to the code

- Text summarizers still need development, and a generic summarizer based on t5-base has good results, but they could be improved

- DeepSpeed is a valuable library for performing inference tasks like translation and summarization

- DeepSpeed, unlike its name, will not make small sample data inference tasks run faster.  Its real magic is with very large tasks like training billion parameter models, or doing inference on very large language data.

We include supplemental (advanced) materials below.  We won't be able to cover them completely in this short workshop, but take a look at them after the workshop and try these out on your own.  They will take much longer to run than our simple examples above.

#### Supplemental (Advanced) Materials

Below are some supplemental exercises you can try on your own. They will not run well on a single GPU cluster, ***thus they are provided as exercises for you to try after the workshop***. These exercises show you how to:

- Use DeepSpeed to train the t5-base model on the CNN/Daily News Dataset
- Train a small sample of protein reads on a billion + parameter Proteomics Model (Rostlab/prot_t5_xl_bfd) which is derived from t5

These are relatively long-running examples (10 to 30 minutes).  The trained models can be used for (i) summarization tasks (t5-base model) and (ii) translation tasks (prot_t5_xl_bfd model).  The examples below are training examples.  The text summarization and text translation examples contained in the upper portion of the tutorial are inference examples.  Once you have your models below trained you can try them out for inference.

**Remember to change any directory links to the output directory of your choice.  We recommend using a dataset.**

#### CNN / Daily News Dataset for Training a Summarization Model

The code below can use ``deepspeed`` to train the t5-small model on the CNN/Daily News dataset.  The dataset has been used for many NLP tasks.  Once the summarizer is trained it can be used for text summarization instead of the regular t5-small model.  The training task will take a very long time to run, so we do not recommend you train it during the workshop.  Its provided as a supplementary exercise you can try later.

In [ ]:
## instantiate the CPP environment on in pytorch

import torch
from torch.utils import cpp_extension

CUDA_HOME = torch.utils.cpp_extension.CUDA_HOME

In [ ]:
%%time

!deepspeed --hostfile /domino/mpi/hosts run_summarization_final.py \
--model_name_or_path t5-small \
--dataset_name cnn_dailymail \
--do_predict \
--dataset_config "3.0.0" --source_prefix "summarize: " \
--output_dir /domino/datasets/local/Downloaded_Datasets --per_device_train_batch_size=4 \
--per_device_eval_batch_size=4 \
--overwrite_output_dir True --predict_with_generate \
--deepspeed ds_config_zero3.json

#### Using your newly trained summarization Model

To run summarization with your newly trained model, follow along with the tutorial on hugging face tutorial here: [Hugging Face Summarization Tutorial](https://huggingface.co/course/chapter7/5?fw=tf).  

Instead choosing a checkpoint from a hugging face model, choose from your newly trained model. You can use something like the following:

```
from transformers import pipeline

model_id = "/link/to/dataset/with-model-checkpoint"
summarizer = pipeline("summarization", model=model_id)

```
Instead of the original tutorial which has you use Hugging Face's repository.  

```
#original code from tutorial

from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=hub_model_id)

```

To test your model you can try the CNN / Daily Mail dataset it is available here: [CNN/Daily Mail News set](https://huggingface.co/datasets/cnn_dailymail)

This is a supplemental exercise to try after the workshop.  It will take some time to train your model, even with ``deepspeed``, so we suggest you have a good, stable internet connection.  We also suggest you try the Hugging Face tutorial as it will be very informative on the process of creating custom trained summarizers.


#### A Proteomics Example

Much of proteomics can be modeled as a 'language' problem.  For example amino acids and DNA nucleotides are represented by strings of letters.  Some laboratories and researchers have taken advantage of this fact and created language / transformer models which are trained on a library of nucleotides or amino acids.  The best known is from the Rostlab. The code below will enable you to train your model on a small sample of amino acids.  For this example the training and validation files are the same -- we would never do that on a real experiment.  This example will take up to 30 minutes or more to train depending on the size of your cluster and the number of GPU cores. Suggested hardware is a GPU with at least p3.2xlarge-V100 cluster of three.  We do not recommend running the code during the workshop as the appropriate hardware is not available.  The model has about 3 billion parameters and was trained on 2.1 billion protein sequences.

For more information on the model you can access its 'card' on Hugging Face here: [Rostlab/pro_t5_xl_bfd](https://huggingface.co/Rostlab/prot_t5_xl_bfd).  For the purposes of this supplemental material we only show you how you could train the model using a small training set of amino acid reads.  The link to the model on Hugging Face has an example of how to use the trained model.

To read up on the model you can go to [ProtTrans: Towards Cracking the Language of Life's Code Through Self-Supervised Deep Learning and High Performance Computing](https://arxiv.org/abs/2007.06225).

In [ ]:
%%time

!deepspeed --hostfile /mnt/domino/hosts run_summarization_t5.py --model_name_or_path "Rostlab/prot_t5_xl_bfd" --output_dir /domino/datasets/local/Output_Directory --overwrite_output_dir True --max_train_samples 5 --max_eval_samples 5 --max_source_length 512 --max_target_length 128 --val_max_target_length 128 --do_train --do_eval --num_train_epochs 1 --warmup_steps 500 --predict_with_generate --save_steps 0 --eval_steps 1 --group_by_length --train_file train_small.csv --test_file train_small.csv --validation_file train_small.csv --text_column text_column --summary_column summary_column --overwrite_cache True --cache_dir /domino/datasets/local/Output_Directory --preprocessing_num_workers 2

